In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
import os
import pickle
import math

from classification.datasets import Dataset_augmented
from classification.utils.audio_student import AudioUtil, Feature_vector_DS
from classification.utils.plots import (
    plot_decision_boundaries,
    plot_specgram,
    show_confusion_matrix,
)
from classification.utils.utils import accuracy

In [2]:
# audio_trimming.py must be run before executing this cell!

# Load the dataset
dataset = Dataset_augmented()
classnames = dataset.list_classes() #['chainsaw', 'fire', 'fireworks', 'gun']
np.random.seed(42)  # For reproducibility

number_audio_files = 280
# splitting it into training and test sets
for i in range(4):
    random_indexes = [np.random.choice(number_audio_files, int(number_audio_files*0.7), replace=False) for _ in range(4)]
    test_indexes = [np.setdiff1d(np.arange(number_audio_files), random_indexes[i]) for i in range(4)]

In [3]:
folder_path = "src/classification/datasets/melspectrograms/"
def fv_generator(Nft_num, nmel_num):
    Nft = Nft_num
    nmel = nmel_num
    # threshold = 0.05 # threshold under which we discard the input
    
    # Computing the mel spectrogram of each audio file and saving in a folder
    n_win_files = np.zeros(4*number_audio_files)
    
    for class_index in range (len(classnames)):
        for audio_index in range(number_audio_files):
            current_sound = dataset[classnames[class_index], audio_index]
            current_audio = AudioUtil.open(current_sound)
            current_audio = AudioUtil.resample(current_audio, 11025)
            
            # we will split the audio into 1 second window, and compute the mel spectrogram of each clip
            n_win = (len(current_audio[0]) // 11025) + 1
            n_win_files[class_index * number_audio_files + audio_index] = int(n_win)
            for window in range(n_win):
                sub_aud = (current_audio[0][window * 11025 :], current_audio[1])
                sub_aud = AudioUtil.pad_trunc(sub_aud, 950)
                sgram = AudioUtil.melspectrogram(sub_aud, Nmel=nmel, Nft=Nft)
                ncol = int(11025 / Nft)
                sgram = sgram[:, :ncol]
                fv = sgram.reshape(-1)
                # saving the mel spectrogram in .npy format
                np.save(folder_path + classnames[class_index] + str(audio_index) + "_" + str(window) + ".npy", fv)
                
    fv_len = len(fv)
    return fv_len, n_win_files
    
    
def pca_generator(training_indexes, pca_comp):
    # Creating all the feature vectors to compute the PCA
    total_number_window_training = np.sum(n_win_files[training_indexes[0]]) + np.sum(n_win_files[training_indexes[1]]) + np.sum(n_win_files[training_indexes[2]]) + np.sum(n_win_files[training_indexes[3]])
    X_train = np.zeros((int(total_number_window_training), int(fv_len)))
    y_train = np.zeros(int(total_number_window_training))
    
    # we will use the indexes to load the mel spectrograms and compute the PCA
    index = 0
    for class_index in range (len(classnames)):
        for audio_index in training_indexes[class_index]:
            for window in range(int(n_win_files[class_index * number_audio_files + audio_index])):
                X_train[index, :] = np.load(folder_path + classnames[class_index] + str(audio_index) + "_" + str(window) + ".npy")
                y_train[index] = class_index
                index += 1
                
    # normalisation before PCA since the amplitude is not a valuable information
    X_train_pca = X_train / np.linalg.norm(X_train, axis=0)
    
    # PCA
    from sklearn.decomposition import PCA
    pca = PCA(n_components=pca_comp)
    pca.fit(X_train_pca)
    
    return pca


In [4]:
def audio_to_fv(class_index, audio_indexes):
    """
    This function takes a class index and a list of audio indexes and returns a matrix of size (len(audio_indexes), n_win_files, fv_len)
    containing the mel spectrogram of each audio file. It also returns a list of labels (which is the same for all the windows of the same audio file)
    """
    X = np.array([np.zeros((int(n_win_files[idx + class_index * number_audio_files]), fv_len)) for idx in audio_indexes], dtype=object)
    label = np.array([[classnames[class_index]] * int(n_win_files[idx + class_index * number_audio_files]) for idx in audio_indexes],dtype=object)
    for i, audio_index in enumerate(audio_indexes):
        for window in range(int(n_win_files[audio_index + class_index * number_audio_files])):
            X[i][window, :] = np.load(folder_path + classnames[class_index] + str(audio_index) + "_" + str(window) + ".npy")
            label[i][window] = classnames[class_index]
    return X, label

In [5]:
import random
import numpy as np

def manual_kfold_random(index_list, n_splits, seed=None):
    """
    Génère des indices pour un k-fold manuel avec répartition aléatoire.

    :param index_list: Liste ou array NumPy des indices totaux
    :param n_splits: Nombre de partitions (doit être inférieur à la taille de la liste)
    :param seed: Optionnel, permet de fixer la graine pour la reproductibilité
    :return: Liste de tuples (learning_set, validation_set)
    """
    assert n_splits < len(index_list), "n_splits doit être inférieur au nombre total d'indices."

    if seed is not None:
        random.seed(seed)  # Fixe la graine pour des résultats reproductibles

    index_list = list(index_list)  # Conversion en liste au cas où c'est un array NumPy
    random.shuffle(index_list)  # Mélange les index aléatoirement

    fold_size = len(index_list) // n_splits
    folds = [index_list[i * fold_size:(i + 1) * fold_size] for i in range(n_splits)]
    
    if len(index_list) % n_splits != 0:  # Gestion des restes
        folds[-1] = list(folds[-1]) + index_list[n_splits * fold_size:]  # Conversion et ajout correct

    split_indices = []
    
    for i in range(n_splits):
        validation_set = folds[i]
        learning_set = [idx for j, fold in enumerate(folds) if j != i for idx in fold]
        split_indices.append((learning_set, validation_set))
    
    return split_indices


def predict_all_probabilities(clf, X, my_fv_len = 400, my_nmel = 20):
    """
    Prédit les probabilités pour tous les feature vectors dans X.

    :param clf: Classifieur entraîné avec une méthode `.predict_proba()`
    :param X: Array de shape (len(classnames), variable, n_win, fv_len)
    :return: Liste des probabilités prédites, réorganisées par classes, échantillons et fenêtres
    """
    len_classes = X.shape[0]  # Nombre de classes
    all_features = []
    indices = []  # Pour garder la trace des indices originaux

    # 1. Extraction des features sous forme de liste
    for class_idx in range(len_classes):
        for sample_idx, feature_matrix in enumerate(X[class_idx]):  # X[class_idx] est de shape (variable, n_win, fv_len)
            num_windows = feature_matrix.shape[0]  # Nombre de fenêtres pour cet échantillon
            all_features.append(feature_matrix)  # Stocke les features
            indices.extend([(class_idx, sample_idx)] * num_windows)  # Associe chaque fenêtre à son (class, sample)

    # 2. Conversion en un array unique pour accélérer la prédiction
    X_flattened = np.vstack(all_features)  # Shape: (total_windows, fv_len)
    X_flattened = X_flattened / np.linalg.norm(X_flattened, axis=0)  # Normal
    
    # if model is not a CNN, we need to reduce the dimensions
    if clf.__class__.__name__ != 'Sequential':
        X_flattened = pca.transform(X_flattened)

    # 3. Prédiction des probabilités en batch
    # if the model is a CNN, we need to reshape the input and to use the predict method
    if clf.__class__.__name__ == 'Sequential':
        new_shape = (X_flattened.shape[0], my_nmel, my_fv_len // my_nmel, 1)
        X_flattened = X_flattened.reshape(new_shape)
        probas_flattened = clf.predict(X_flattened)  # Sortie: (total_windows, n_classes)
    else:
        probas_flattened = clf.predict_proba(X_flattened)  # Sortie: (total_windows, n_classes)

    # 4. Réorganisation des probabilités
    probabilities = [[] for _ in range(len_classes)]
    for (class_idx, sample_idx), proba in zip(indices, probas_flattened):
        if len(probabilities[class_idx]) <= sample_idx:
            probabilities[class_idx].append([])  # Assure que la liste existe
        probabilities[class_idx][sample_idx].append(proba)  # Ajoute la probabilité pour chaque fenêtre

    return probabilities  # Liste [classes][samples][windows][probs]

In [6]:
def decision_naive(probs):
    """Règle naïve : Choisir la classe avec la probabilité la plus élevée."""
    return classnames[np.argmax(probs) % len(classnames)]

def decision_majority(probs):
    """Règle du vote majoritaire : Choisir la classe avec le plus grand nombre de votes."""
    votes = np.argmax(probs, axis=1)
    count = np.bincount(votes)
    max_indices = np.where(count == count.max())[0]
    return classnames[max_indices[0]]  # En cas d'égalité, on prend la première classe

def decision_weighted(probs):
    """Règle du vote pondéré : Choisir la classe avec la somme maximale des probabilités."""
    sum_probs = np.sum(probs, axis=0)
    return classnames[np.argmax(sum_probs)]

def decision_maxlikelihood(probs):
    """Règle du maximum de vraisemblance : Choisir la classe avec le produit maximal des probabilités."""
    prod_probs = np.prod(probs, axis=0)
    return classnames[np.argmax(prod_probs)]

# Application de la règle de décision à chaque échantillon d'un audio
def apply_decision_rules(probabilities, decision_method):
    """
    Applique une règle de décision sur toutes les fenêtres d'un échantillon.

    :param probabilities: Probabilités des fenêtres pour un échantillon (n_win, n_classes)
    :param decision_method: Méthode de décision à utiliser parmi 'naive', 'majority', 'weighted', 'maxlikelihood' [0, 1, 2, 3]
    :return: Classe prédite pour l'échantillon
    """
    if decision_method == 0:
        return decision_naive(probabilities)
    elif decision_method == 1:
        return decision_majority(probabilities)
    elif decision_method == 2:
        return decision_weighted(probabilities)
    elif decision_method == 3:
        return decision_maxlikelihood(probabilities)
    else:
        raise ValueError("Unknown decision method: " + decision_method)
    
dict_decision_methods = {
    0: "Naive",
    1: "Majority",
    2: "Weighted",
    3: "MaxLikelihood"
}


def predict_for_all_samples(probabilities, decision_method):
    """
    Applique une règle de décision sur tous les échantillons (classes, samples) du dataset.

    :param probabilities: Liste des probabilités pour chaque fenêtre de chaque échantillon.
    :param decision_method: Méthode de décision ('naive', 'majority', 'weighted', 'maxlikelihood')
    :return: Liste des classes prédites pour chaque échantillon
    """
    n_classes = len(probabilities)
    n_samples = len(probabilities[0])
    predictions = []

    for class_idx in range(n_classes):
        class_predictions = []
        for sample_idx in range(n_samples):
            # Probabilités des fenêtres pour un échantillon donné
            probs = probabilities[class_idx][sample_idx]
            # Applique la règle de décision sur l'échantillon
            predicted_class = apply_decision_rules(probs, decision_method)
            class_predictions.append(predicted_class)
        predictions.append(class_predictions)

    return predictions

import numpy as np

def compute_accuracy(predictions, y_val):
    """
    Calcule l'accuracy en comparant les prédictions avec les labels réels.

    :param predictions: Liste des prédictions pour chaque échantillon et chaque classe
                        (n_classes, n_samples) - chaque élément est la classe prédite pour un échantillon.
    :param y_val: Array des labels réels pour chaque échantillon (n_classes, n_samples)
                  Chaque élément est la classe réelle de cet échantillon.
    :return: L'accuracy du modèle en pourcentage
    """
    correct_predictions = 0
    total_predictions = 0

    n_classes = len(predictions)

    for class_idx in range(n_classes):
        for sample_idx in range(len(predictions[class_idx])):
            # Comparer la prédiction avec le label réel
            predicted_class = predictions[class_idx][sample_idx]
            true_class = y_val[class_idx][sample_idx][0]
            
            # Si la prédiction est correcte, on incrémente le compteur
            if predicted_class == true_class:
                correct_predictions += 1
            total_predictions += 1

    # Calculer l'accuracy
    accuracy = correct_predictions / total_predictions * 100
    return accuracy



In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models

def create_cnn(input_shape, num_classes):
    H, W, C = input_shape  # Hauteur, Largeur, Canaux

    # Définition des tailles de kernels basées sur la taille de l'image
    k1 = (max(1, H // 6), max(1, W // 6))  # Premier kernel (minimum 1x1)
    k2 = (max(1, H // 7), max(1, W // 7))  # Deuxième kernel
    k3 = (max(1, H // 8), max(1, W // 8))  # Troisième kernel
    k4 = (max(1, H // 9), max(1, W // 9))  # Quatrième kernel
    k5 = (max(1, H // 10), max(1, W // 10)) # Cinquième kernel
    
    model = models.Sequential()

    # L1: Convolution + ReLU + MaxPooling
    model.add(layers.Conv2D(24, k1, activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    # L2: Convolution + ReLU + Padding + MaxPooling
    model.add(layers.Conv2D(48, k2, activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    # L3: Convolution + ReLU + Padding (No MaxPooling)
    model.add(layers.Conv2D(48, k3, activation='relu', padding='same'))

    # L4: Convolution + ReLU + Padding (No MaxPooling)
    model.add(layers.Conv2D(60, k4, activation='relu', padding='same'))

    # L5: Convolution + ReLU + Padding (No MaxPooling)
    model.add(layers.Conv2D(72, k5, activation='relu', padding='same'))

    # Flatten the output for the dense layers
    model.add(layers.Flatten())

    # Déterminer dynamiquement la taille du premier dense
    final_dim = (H // 4) * (W // 4) * 72  # Taille après convolutions et max-pooling
    dense_units = max(64, final_dim // 10)  # Éviter un nombre trop petit
    
    # L6: Fully Connected + ReLU + Dropout
    model.add(layers.Dense(dense_units, activation='relu'))
    model.add(layers.Dropout(0.5))

    # L7: Output Layer with Softmax
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

In [ ]:
# we will try training in a different way now, basically we want that the models fits to melspectrogram of the audio files of a subset of the training set
# but for the validation, we want to evaluate it on a sequence of melspectrogram from the complementary subset of the training set

# Model training
# we will test three models, CNN, SVM and Random Forest
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
# TO DO : implement the CNN model

# 5-fold cross validation
from sklearn.model_selection import KFold
n_splits = 5
seed = 42

kfold_indexes = [[], [], [], []]
for class_index in range (len(classnames)):
    kfold_indexes[class_index] = manual_kfold_random(random_indexes[class_index], n_splits, seed=seed)

# hyperparameters of the spectrogram
Nft_list = [256, 512, 1024]
nmel_list = [15, 20, 30]
pca_comp_list = [0.25, 0.5, 0.75, 1]

# Random Forest
n_estimators = [100]
max_depth = [20]
min_samples_split = [5]

results_RF = pd.DataFrame(columns=["model", "n_estimators", "max_depth", "min_samples_split", "accuracy", "decision_method", "Nft", "nmel", "pca_comp"])

for Nft in Nft_list:
    for nmel in nmel_list:
        for pca_comp in pca_comp_list:
            fv_len, n_win_files = fv_generator(Nft, nmel)
            pca = pca_generator(random_indexes, int(pca_comp*fv_len))
            for n in n_estimators:
                for d in max_depth:
                    for s in min_samples_split:
                        clf = RandomForestClassifier(n_estimators=n, max_depth=d, min_samples_split=s)
                        accuracies = np.zeros(len(classnames))
                        
                        # creating the indexes for each split
                        kfold_indexes = [[], [], [], []]
                        for class_index in range (len(classnames)):
                            kfold_indexes[class_index] = manual_kfold_random(random_indexes[class_index], n_splits, seed=seed)
                            
                        # kfold cross validation
                        for split in range(n_splits):
                            X_train = np.empty(len(classnames), dtype=object)
                            y_train = np.empty(len(classnames), dtype=object)
                            X_val = np.empty(len(classnames), dtype=object)
                            y_val = np.empty(len(classnames), dtype=object)
                            for class_index in range (len(classnames)):
                                X_train[class_index], y_train[class_index] = audio_to_fv(class_index, kfold_indexes[class_index][split][0])
                                X_val[class_index], y_val[class_index] = audio_to_fv(class_index, kfold_indexes[class_index][split][1])
                                
                            # Training
                            X_train = np.vstack([sample for class_samples in X_train for sample in class_samples])
                            y_train = np.hstack([np.repeat(label, len(samples)) for samples, labels in zip(X_train, y_train) for label in labels])
                            y_train = y_train[::fv_len]
                            X_train = X_train/np.linalg.norm(X_train, axis=0)
                            X_train = pca.transform(X_train)
                            clf.fit(X_train, y_train)
                            
                            # Validation
                            probas = predict_all_probabilities(clf, X_val)
                            for decision_index in range(4):
                                accuracies[decision_index] += compute_accuracy(predict_for_all_samples(probas,decision_index), y_val)
                                
                        # averaging the accuracies
                        accuracies /= n_splits
                        for decision_index in range(4):
                            results_RF = results_RF._append({"model": "Random Forest", "n_estimators": n, "max_depth": d, "min_samples_split": s, "accuracy": accuracies[decision_index], "decision_method": dict_decision_methods[decision_index], "Nft": Nft, "nmel": nmel, "pca_comp": pca_comp}, ignore_index=True)
                    print(results_RF.loc[results_RF["accuracy"].idxmax()])

# Save the results in .csv
results_RF.to_csv("results_RF.csv")
                
                    

In [ ]:
# CNN

# 5-fold cross validation
from sklearn.model_selection import KFold
n_splits = 5
seed = 42

# hyperparameters of the spectrogram
Nft_list = [128, 256, 512, 1024, 2048]
nmel_list = [10, 15, 20, 25, 30]

# CNN hyperparameters
epochs_list = [10]
batch_size_list = [32]

results_CNN = pd.DataFrame(columns=["model", "epochs", "batch_size", "accuracy", "decision_method", "Nft", "nmel"])

for Nft in Nft_list:
    for nmel in nmel_list:
        fv_len, n_win_files = fv_generator(Nft, nmel)
        for e in epochs_list:
            for b in batch_size_list:
                for decision_index in range(3,4):
                    accuracies = np.zeros(len(classnames))
                    
                    # creating the indexes for each split
                    kfold_indexes = [[], [], [], []]
                    for class_index in range (len(classnames)):
                        kfold_indexes[class_index] = manual_kfold_random(random_indexes[class_index], n_splits, seed=seed)
                        
                    # kfold cross validation
                    for split in range(n_splits):
                        X_train = np.empty(len(classnames), dtype=object)
                        y_train = np.empty(len(classnames), dtype=object)
                        X_val = np.empty(len(classnames), dtype=object)
                        y_val = np.empty(len(classnames), dtype=object)
                        for class_index in range (len(classnames)):
                            X_train[class_index], y_train[class_index] = audio_to_fv(class_index, kfold_indexes[class_index][split][0])
                            X_val[class_index], y_val[class_index] = audio_to_fv(class_index, kfold_indexes[class_index][split][1])
                            
                        # Training
                        X_train = np.vstack([sample for class_samples in X_train for sample in class_samples])
                        y_train = np.hstack([np.repeat(label, len(samples)) for samples, labels in zip(X_train, y_train) for label in labels])
                        y_train = y_train[::fv_len]
                        X_train = X_train/np.linalg.norm(X_train, axis=0)
                        
                        # Reshape the data for the CNN
                        new_shape = (X_train.shape[0], nmel, fv_len // nmel, 1)
                        X_train = X_train.reshape(new_shape)
                        
                        # Create the CNN model
                        model = create_cnn((nmel, fv_len // nmel, 1), len(classnames))
                        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
                        
                        # One-hot encoding of the labels
                        y_train = pd.get_dummies(y_train).values
                        
                        # Training
                        model.fit(X_train, y_train, epochs=e, batch_size=b)
                        
                        # Validation
                        probas = predict_all_probabilities(model, X_val, fv_len, nmel)
                        accuracies[decision_index] += compute_accuracy(predict_for_all_samples(probas,decision_index), y_val)
                        
                    # averaging the accuracies
                    accuracies /= n_splits
                    for decision_index in range(4):
                        results_CNN = results_CNN._append({"model": "CNN", "epochs": e, "batch_size": b, "accuracy": accuracies[decision_index], "decision_method": dict_decision_methods[decision_index], "Nft": Nft, "nmel": nmel}, ignore_index=True)
                print(results_CNN.loc[results_CNN["accuracy"].idxmax()])
                    
# Save the results in .csv
results_CNN.to_csv("results_CNN.csv")

C:\Users\vicky\Documents\LELEC210X_GROUP_E\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.4853 - loss: 1.1355
Epoch 2/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.6258 - loss: 0.8611
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.6955 - loss: 0.7803
Epoch 4/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7685 - loss: 0.6474
Epoch 5/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7803 - loss: 0.6008
Epoch 6/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.8145 - loss: 0.5362
Epoch 7/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.8266 - loss: 0.4879
Epoch 8/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8177 - loss: 0.5044
Epoch 9/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8549 - loss: 0.4293
Epoch 10/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8546 - loss: 0.4165
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step


C:\Users\vicky\Documents\LELEC210X_GROUP_E\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.5247 - loss: 1.0890
Epoch 2/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6377 - loss: 0.8777
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6500 - loss: 0.7981
Epoch 4/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7358 - loss: 0.6879
Epoch 5/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7379 - loss: 0.6778
Epoch 6/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.7750 - loss: 0.5788
Epoch 7/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8061 - loss: 0.5280
Epoch 8/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8269 - loss: 0.4461
Epoch 9/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.8190 - loss: 0.4702
Epoch 10/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.8681 - loss: 0.3671
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


C:\Users\vicky\Documents\LELEC210X_GROUP_E\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

75/75 ━━━━━━━━━━━━━━━━━━━━ 7s 57ms/step - accuracy: 0.5256 - loss: 1.1254
Epoch 2/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 54ms/step - accuracy: 0.6582 - loss: 0.8529
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 54ms/step - accuracy: 0.7295 - loss: 0.6753
Epoch 4/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 5s 62ms/step - accuracy: 0.7959 - loss: 0.5279
Epoch 5/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 54ms/step - accuracy: 0.8310 - loss: 0.4589
Epoch 6/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 56ms/step - accuracy: 0.8527 - loss: 0.3903
Epoch 7/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - accuracy: 0.8791 - loss: 0.3189
Epoch 8/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - accuracy: 0.9001 - loss: 0.2603
Epoch 9/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - accuracy: 0.9082 - loss: 0.2362
Epoch 10/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step - accuracy: 0.9010 - loss: 0.2779
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step
Epoch 1/10


C:\Users\vicky\Documents\LELEC210X_GROUP_E\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


75/75 ━━━━━━━━━━━━━━━━━━━━ 6s 51ms/step - accuracy: 0.5372 - loss: 1.1337
Epoch 2/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step - accuracy: 0.6364 - loss: 0.9249
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 49ms/step - accuracy: 0.7528 - loss: 0.6667
Epoch 4/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 52ms/step - accuracy: 0.8152 - loss: 0.5091
Epoch 5/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 51ms/step - accuracy: 0.8356 - loss: 0.4359
Epoch 6/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 53ms/step - accuracy: 0.8842 - loss: 0.3240
Epoch 7/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step - accuracy: 0.9022 - loss: 0.2549
Epoch 8/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 53ms/step - accuracy: 0.9021 - loss: 0.2876
Epoch 9/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 5s 60ms/step - accuracy: 0.9103 - loss: 0.2361
Epoch 10/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 5s 60ms/step - accuracy: 0.9149 - loss: 0.2312
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step
model                        CNN
epochs                        10
batch_size                    32
accuracy             

C:\Users\vicky\Documents\LELEC210X_GROUP_E\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 8s 75ms/step - accuracy: 0.5505 - loss: 1.0888
Epoch 2/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 5s 70ms/step - accuracy: 0.6686 - loss: 0.8320
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 5s 71ms/step - accuracy: 0.7807 - loss: 0.5882
Epoch 4/10
49/75 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step - accuracy: 0.8047 - loss: 0.6226

In [ ]:
# SVM

# 5-fold cross validation
from sklearn.model_selection import KFold
n_splits = 5
seed = 42

# hyperparameters of the spectrogram
Nft_list = [512]
nmel_list = [20]

# SVM hyperparameters
C_list = [1]
kernel_list = ['rbf']
gamma_list = ['scale']

results_SVM = pd.DataFrame(columns=["model", "C", "kernel", "gamma", "accuracy", "decision_method", "Nft", "nmel"])

for Nft in Nft_list:
    for nmel in nmel_list:
        fv_len, n_win_files = fv_generator(Nft, nmel)
        for c in C_list:
            for k in kernel_list:
                for g in gamma_list:
                    clf = SVC(C=c, kernel=k, gamma=g)
                    accuracies = np.zeros(len(classnames))
                    
                    # creating the indexes for each split
                    kfold_indexes = [[], [], [], []]
                    for class_index in range (len(classnames)):
                        kfold_indexes[class_index] = manual_kfold_random(random_indexes[class_index], n_splits, seed=seed)
                        
                    # kfold cross validation
                    for split in range(n_splits):
                        X_train = np.empty(len(classnames), dtype=object)
                        y_train = np.empty(len(classnames), dtype=object)
                        X_val = np.empty(len(classnames), dtype=object)
                        y_val = np.empty(len(classnames), dtype=object)
                        for class_index in range (len(classnames)):
                            X_train[class_index], y_train[class_index] = audio_to_fv(class_index, kfold_indexes[class_index][split][0])
                            X_val[class_index], y_val[class_index] = audio_to_fv(class_index, kfold_indexes[class_index][split][1])
                            
                        # Training
                        X_train = np.vstack([sample for class_samples in X_train for sample in class_samples])
                        y_train = np.hstack([np.repeat(label, len(samples)) for samples, labels in zip(X_train, y_train) for label in labels])
                        y_train = y_train[::fv_len]
                        X_train = X_train/np.linalg.norm(X_train, axis=0)
                        X_train = pca.transform(X_train)
                        clf.fit(X_train, y_train)
                        
                        # Validation
                        probas = predict_all_probabilities(clf, X_val)
                        for decision_index in range(4):
                            accuracies[decision_index] += compute_accuracy(predict_for_all_samples(probas,decision_index), y_val)
                        
                    # averaging the accuracies
                    accuracies /= n_splits
                    for decision_index in range(4):
                        results_SVM = results_SVM._append({"model": "SVM", "C": c, "kernel": k, "gamma": g, "accuracy": accuracies[decision_index], "decision_method": dict_decision_methods[decision_index], "Nft":
Nft, "nmel": nmel}, ignore_index=True)
                        
                    print(results_SVM.loc[results_SVM["accuracy"].idxmax()])
                    
# Save the results in .csv
results_SVM.to_csv("results_SVM.csv")


In [ ]:
# test set evaluation
# for now we will just use random forest with basic parameters

# hyperparameters of the spectrogram
Nft = 512
nmel = 20

# other hyperparameters
pca_comp = 0.5
best_decision = 3

model = RandomForestClassifier(n_estimators=100, max_depth=20, min_samples_split=5)
fv_len, n_win_files = fv_generator(Nft, nmel)
pca = pca_generator(random_indexes, int(pca_comp*fv_len))

# Training
X_train = np.empty(len(classnames), dtype=object)
y_train = np.empty(len(classnames), dtype=object)
for class_index in range (len(classnames)):
    X_train[class_index], y_train[class_index] = audio_to_fv(class_index, random_indexes[class_index])
X_train = np.vstack([sample for class_samples in X_train for sample in class_samples])
y_train = np.hstack([np.repeat(label, len(samples)) for samples, labels in zip(X_train, y_train) for label in labels])
y_train = y_train[::fv_len]
X_train = X_train/np.linalg.norm(X_train, axis=0)
X_train = pca.transform(X_train)
model.fit(X_train, y_train)

# Testing
X_test = np.empty(len(classnames), dtype=object)
y_test = np.empty(len(classnames), dtype=object)
for class_index in range (len(classnames)):
    X_test[class_index], y_test[class_index] = audio_to_fv(class_index, test_indexes[class_index])
X_test = np.vstack([sample for class_samples in X_test for sample in class_samples])
y_test = np.hstack([np.repeat(label, len(samples)) for samples, labels in zip(X_test, y_test) for label in labels])
y_test = y_test[::fv_len]
X_test = X_test/np.linalg.norm(X_test, axis=0)
X_test = pca.transform(X_test)
probas = predict_all_probabilities(model, X_test)
predictions = predict_for_all_samples(probas, best_decision)
accuracy = compute_accuracy(predictions, y_test)
print(accuracy)

In [ ]:
# OLD Model training
# we will test three models, CNN, SVM and Random Forest
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
# TO DO : implement the CNN model

# 5-fold cross validation
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

# Random Forest
n_estimators = [10, 50, 100, 200]
max_depth = [5, 10, 20, 50, 100]
min_samples_split = [2, 5, 10, 15]

results_RF = pd.DataFrame(columns=["model", "n_estimators", "max_depth", "min_samples_split", "accuracy"])

for n in n_estimators:
    for d in max_depth:
        for s in min_samples_split:
            clf = RandomForestClassifier(n_estimators=n, max_depth=d, min_samples_split=s)
            acc = 0
            for train_index, test_index in kf.split(X_train):
                X_train_kf, X_test_kf = X_train[train_index], X_train[test_index]
                y_train_kf, y_test_kf = y_train[train_index], y_train[test_index]
                clf.fit(X_train_kf, y_train_kf)
                acc += accuracy(y_test_kf, clf.predict(X_test_kf))
            acc /= 5
            results_RF = results_RF._append({"model": "Random Forest", "n_estimators": n, "max_depth": d, "min_samples_split": s, "accuracy": acc}, ignore_index=True)
        print(results_RF.loc[results_RF["accuracy"].idxmax()])
        
# save the results in a .csv file
results_RF.to_csv("results_RF.csv")


# SVM
kernel = ["rbf"]
C = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
gamma = ["scale", "auto", 0.1, 0.5, 1, 5, 10]

# the results will be saved in a pandas dataframe
results_SVM = pd.DataFrame(columns=["model", "kernel", "C", "gamma", "accuracy"])

for k in kernel:
    for c in C:
        for g in gamma:
            clf = SVC(kernel=k, C=c, gamma=g)
            acc = 0
            for train_index, test_index in kf.split(X_train):
                X_train_kf, X_test_kf = X_train[train_index], X_train[test_index]
                y_train_kf, y_test_kf = y_train[train_index], y_train[test_index]
                clf.fit(X_train_kf, y_train_kf)
                acc += accuracy(y_test_kf, clf.predict(X_test_kf))
            acc /= 5
            results_SVM = results_SVM._append({"model": "SVM", "kernel": k, "C": c, "gamma": g, "accuracy": acc}, ignore_index=True)
        print(results_SVM.loc[results_SVM["accuracy"].idxmax()])

# save the results in a .csv file
results_SVM.to_csv("results_SVM.csv")